In [ ]:
## DEFINE VARIABLES ##
hs_url = "http://hydroportal.cuahsi.org/para_la_naturaleza/cuahsi_1_1.asmx?WSDL"
hs_name = "Para la Naturaleza"

In [10]:
!pip install xmltodict
!pip install pandas
!pip install numpy
!pip install owslib
!pip install suds-py3
!pip install folium

In [12]:
##IMPORT THE DEPENDENCIES
import xmltodict
import json
import pandas as pd
import numpy as np
import folium
import xml.etree.ElementTree as ET
from owslib.waterml.wml11 import WaterML_1_1 as wml11
from suds.client import Client  # For parsing WaterML/XML
from json import dumps, loads
from datetime import datetime

In [47]:
def parseJSON(json):
    hs_sites = []
    sites_object = None
    # This is to handle the WMO la Plata endpoints ##

    if "Site" in json:
        sites_object = json['Site']
        # If statement is executed for multiple sites within the HydroServer, if there is a single site then it goes to the else statement
        # Parse through the HydroServer and each site with its metadata as a
        # dictionary object to the hs_sites list
        if type(sites_object) is list:
            for site in sites_object:
                hs_json = {}
                latitude = site['Latitude']
                longitude = site['Longitude']
                site_name = site['SiteName']
                site_name = site_name.encode("utf-8")
                network = site["servURL"]
                sitecode = site['SiteCode']

                hs_json["sitename"] = site_name.decode("UTF-8")
                hs_json["latitude"] = latitude
                hs_json["longitude"] = longitude
                hs_json["sitecode"] = sitecode
                hs_json["network"] = network
                hs_json["service"] = "SOAP"
                hs_sites.append(hs_json)
        else:
            hs_json = {}
            latitude = site['Latitude']
            longitude = site['Longitude']
            site_name = site['SiteName']
            site_name = site_name.encode("utf-8")
            network = site["servURL"]
            sitecode = site['SiteCode']

            hs_json["sitename"] = site_name.decode("UTF-8")
            hs_json["latitude"] = latitude
            hs_json["longitude"] = longitude
            hs_json["sitecode"] = sitecode
            hs_json["network"] = network
            hs_json["service"] = "SOAP"
            hs_sites.append(hs_json)

    if "sitesResponse" in json:
        sites_object = json['sitesResponse']['site']

        # If statement is executed for multiple sites within the HydroServer, if there is a single site then it goes to the else statement
        # Parse through the HydroServer and each site with its metadata as a
        # dictionary object to the hs_sites list
        if type(sites_object) is list:
            for site in sites_object:
                hs_json = {}
                latitude = site['siteInfo']['geoLocation'][
                    'geogLocation']['latitude']
                longitude = site['siteInfo']['geoLocation'][
                    'geogLocation']['longitude']
                site_name = site['siteInfo']['siteName']
                site_name = site_name.encode("utf-8")
                network = site['siteInfo']['siteCode']["@network"]
                sitecode = site['siteInfo']['siteCode']["#text"]

                hs_json["sitename"] = site_name.decode("UTF-8")
                hs_json["latitude"] = latitude
                hs_json["longitude"] = longitude
                hs_json["sitecode"] = sitecode
                hs_json["network"] = network
                hs_json["service"] = "SOAP"
                hs_sites.append(hs_json)
        else:
            hs_json = {}
            latitude = sites_object['siteInfo'][
                'geoLocation']['geogLocation']['latitude']
            longitude = sites_object['siteInfo'][
                'geoLocation']['geogLocation']['longitude']
            site_name = sites_object['siteInfo']['siteName']
            site_name = site_name.encode("utf-8")
            network = sites_object['siteInfo']['siteCode']["@network"]
            sitecode = sites_object['siteInfo']['siteCode']["#text"]

            hs_json["sitename"] = site_name.decode("UTF-8")
            hs_json["latitude"] = latitude
            hs_json["longitude"] = longitude
            hs_json["sitecode"] = sitecode
            hs_json["network"] = network
            hs_json["service"] = "SOAP"
            hs_sites.append(hs_json)

    return hs_sites


def add_hs(url):
    return_obj = {}
    client = Client(url, timeout= 500)
    # True Extent is on and necessary if the user is trying to add USGS or
    # Get a list of all the sites and their respective lat lon.
    sites = client.service.GetSites('[:]')
    sites_json={}
    if isinstance(sites, str):
        print("here")
        sites_dict = xmltodict.parse(sites)
        sites_json_object = json.dumps(sites_dict)
        sites_json = json.loads(sites_json_object)
    else:
        sites_json_object = suds_to_json(sites)
        sites_json = json.loads(sites_json_object)

    sites_object = parseJSON(sites_json)
 
    return_obj['siteInfo'] = sites_object
    return return_obj

def get_lats_lngs(objSites):
    sites = objSites['siteInfo']
    sitesLoc = []
    names = []
    print(sites)
    for site in sites:
      print(site)
      sitesLoc.append([float(site['latitude']) , float(site['longitude'])])
      names.append(site['sitename'])
    return [sitesLoc,names]


In [48]:
sitesResponse = add_hs(hs_url)
print(sites)
sitesLocations,sitesNames = get_lats_lngs(sitesResponse)


here
{'siteInfo': '[{"sitename": "R\\u00edo Toro Negro", "latitude": "18.28559", "longitude": "-66.4903", "sitecode": "Rio_Toro_Negro", "network": "Para_La_Naturaleza", "service": "SOAP"}, {"sitename": "Quebrada Batista", "latitude": "18.19699", "longitude": "-66.32992", "sitecode": "Quebrada_Batista", "network": "Para_La_Naturaleza", "service": "SOAP"}, {"sitename": "R\\u00edo Grande de Manat\\u00ed", "latitude": "18.2144", "longitude": "-66.28665", "sitecode": "Rio_Grande_de_Manati", "network": "Para_La_Naturaleza", "service": "SOAP"}]'}
[{'sitename': 'Río Toro Negro', 'latitude': '18.28559', 'longitude': '-66.4903', 'sitecode': 'Rio_Toro_Negro', 'network': 'Para_La_Naturaleza', 'service': 'SOAP'}, {'sitename': 'Quebrada Batista', 'latitude': '18.19699', 'longitude': '-66.32992', 'sitecode': 'Quebrada_Batista', 'network': 'Para_La_Naturaleza', 'service': 'SOAP'}, {'sitename': 'Río Grande de Manatí', 'latitude': '18.2144', 'longitude': '-66.28665', 'sitecode': 'Rio_Grande_de_Manati', 

In [55]:
# Create a map using Stamen Terrain, centered on Boulder, CO
m = folium.Map(height=800,location=[40.0150, -105.2705], 
              tiles = 'Stamen Terrain')

# Add marker for Boulder, CO
print(sitesLocations)
for ind in range(len(sitesLocations)):
  folium.Marker(
      location= sitesLocations[ind], # coordinates for the marker (Earth Lab at CU Boulder)
      popup= sitesNames[ind], # pop-up label for the marker
      icon=folium.Icon()
  ).add_to(m)

# Display m
m.fit_bounds(sitesLocations)
m

[[18.28559, -66.4903], [18.19699, -66.32992], [18.2144, -66.28665]]
